## Demonsration of `jitr.reaction.channels`, building the channels for collision of a projectile on a target, including coupling to excited states.

In [18]:
from fractions import Fraction
import numpy as np
from IPython.display import Latex, Math, display

In [2]:
import jitr.reactions.channels as ch
from jitr.structure import Level, Parity
from jitr.utils.angular_momentum import (
    format_jbasis_channel_latex,
    format_sbasis_channel_latex,
    triangle_rule,
)

## Set up the system
We can set the spin and parity of the projectile and target ground and excited states like so:

In [3]:
# add some excited states to the target
target = [
    Level(E=0, I=0, pi=Parity.positive),
    Level(E=1, I=2, pi=Parity.positive),
    Level(E=2, I=4, pi=Parity.positive),
]
# just the ground state for the projectile
projectile = [
    Level(E=0, I=Fraction(1, 2), pi=Parity.positive),
]

In [4]:
lmax = 10

Jtot_max = lmax + projectile[0].I + target[2].I  # highest total angular momentum
Jtot_min = abs(projectile[0].I - target[0].I)  # lowest (lmin = 0)
n_partial_waves = int(Jtot_max - Jtot_min)
print(n_partial_waves)

14


In [5]:
print(Jtot_min)

1/2


In [6]:
print(Jtot_max)

29/2


## Construct all $J$, $\pi$ states
Each of these $J$, $\pi$ pairs represents a system of coupled partial waves

In [7]:
Jpi = [
    [(Jtot_min + n, Parity.positive) for n in range(n_partial_waves)],
    [(Jtot_min + n, Parity.negative) for n in range(n_partial_waves)],
]

## Look at the systems for a few  $J$, $\pi$ pairs in the $J = L + I_p$ basis


In [8]:
# positive total parity states
for J, pi in Jpi[0][0:4]:
    display(Math(f"J={J}"))
    channels = ch.build_all_channels(
        J, pi, projectile, target, build_channels=ch.build_channels_2body_jbasis
    )
    coupled_channel_strings = []
    for channel in channels:
        j = channel["j"]
        l = channel["l"]
        coupled_channel_strings.append(f"{format_jbasis_channel_latex(l,J,j)}")
    lds = [f"{int(x)}" for x in ch.spin_orbit_coupling(channels)]
    display(Math("(" + ", \,".join(coupled_channel_strings) + ")"))
    display(Math("< l \cdot s > = (" + ", \,".join(lds) + ")"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [9]:
# negative total parity states
for J, pi in Jpi[1][0:4]:
    display(Math(f"J={J}"))
    channels = ch.build_all_channels(
        J, pi, projectile, target, build_channels=ch.build_channels_2body_jbasis
    )
    coupled_channel_strings = []
    for channel in channels:
        j = channel["j"]
        l = channel["l"]
        coupled_channel_strings.append(f"{format_jbasis_channel_latex(l,J,j)}")
    lds = [f"{int(x)}" for x in ch.spin_orbit_coupling(channels)]
    display(Math("(" + ", \,".join(coupled_channel_strings) + ")"))
    display(Math("< l \cdot s > = (" + ", \,".join(lds) + ")"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# How about the $S = I_p + I_t$ basis?
We won't calculate the spin-orbit because it is not diagonal in the $S$ basis

In [48]:
# positive total parity states
for J, pi in Jpi[0][0:4]:
    display(Math(f"J={J}"))
    channels = ch.build_all_channels(
        J, pi, projectile, target, build_channels=ch.build_channels_2body_sbasis
    )
    coupled_channel_strings = []
    for channel in channels:
        s = channel["s"]
        l = channel["l"]
        coupled_channel_strings.append(f"{format_sbasis_channel_latex(l,J,s)}")
    display(Math("(" + ", \,".join(coupled_channel_strings) + ")"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [49]:
# negative total parity states
for J, pi in Jpi[1][0:4]:
    display(Math(f"J={J}"))
    channels = ch.build_all_channels(
        J, pi, projectile, target, build_channels=ch.build_channels_2body_sbasis
    )
    coupled_channel_strings = []
    for channel in channels:
        s = channel["s"]
        l = channel["l"]
        coupled_channel_strings.append(f"{format_sbasis_channel_latex(l,J,s)}")
    display(Math("(" + ", \,".join(coupled_channel_strings) + ")"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Converting between $J$ and $S$ bases
Let's take the channels making up the $J=1/2, \pi=+1$ system and demonstrate the unitary transformation between bases

In [88]:
Jtot = Fraction(1,2)
pi = Parity.positive
ch_jbasis = channels = ch.build_all_channels(
    Jtot,
    pi,
    projectile,
    target,
    build_channels=ch.build_channels_2body_jbasis,
)
ch_sbasis = channels = ch.build_all_channels(
    Jtot,
    pi,
    projectile,
    target,
    build_channels=ch.build_channels_2body_sbasis,
)

In [105]:
ch_jbasis

array([(0, Fraction(1, 2), 0., Fraction(1, 2), <Parity.positive: True>, 0., 0, <Parity.positive: True>),
       (2, Fraction(3, 2), 0., Fraction(1, 2), <Parity.positive: True>, 1., 2, <Parity.positive: True>),
       (2, Fraction(5, 2), 0., Fraction(1, 2), <Parity.positive: True>, 1., 2, <Parity.positive: True>),
       (4, Fraction(7, 2), 0., Fraction(1, 2), <Parity.positive: True>, 2., 4, <Parity.positive: True>),
       (4, Fraction(9, 2), 0., Fraction(1, 2), <Parity.positive: True>, 2., 4, <Parity.positive: True>)],
      dtype=[('l', '<i8'), ('j', 'O'), ('Ex_p', '<f8'), ('Ip', 'O'), ('pi_p', 'O'), ('Ex_t', '<f8'), ('It', 'O'), ('pi_t', 'O')])

In [106]:
ch_sbasis

array([(0, Fraction(1, 2), 0., Fraction(1, 2), <Parity.positive: True>, 0., 0, <Parity.positive: True>),
       (2, Fraction(3, 2), 0., Fraction(1, 2), <Parity.positive: True>, 1., 2, <Parity.positive: True>),
       (2, Fraction(5, 2), 0., Fraction(1, 2), <Parity.positive: True>, 1., 2, <Parity.positive: True>),
       (4, Fraction(7, 2), 0., Fraction(1, 2), <Parity.positive: True>, 2., 4, <Parity.positive: True>),
       (4, Fraction(9, 2), 0., Fraction(1, 2), <Parity.positive: True>, 2., 4, <Parity.positive: True>)],
      dtype=[('l', '<i8'), ('s', 'O'), ('Ex_p', '<f8'), ('Ip', 'O'), ('pi_p', 'O'), ('Ex_t', '<f8'), ('It', 'O'), ('pi_t', 'O')])

In [107]:
U_s2j = ch.sbasis_to_jbasis_conversion_matrix(Jtot, ch_sbasis, ch_jbasis)

In [108]:
U_s2j.shape

(5, 5)

In [109]:
U_s2j.conj().T @ U_s2j

array([[ 1.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j],
       [ 0.        +0.j,  0.68      +0.j, -0.08      +0.j,
         0.        +0.j,  0.        +0.j],
       [ 0.        +0.j, -0.08      +0.j,  1.48      +0.j,
         0.        +0.j,  0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.80246914+0.j, -0.02469136+0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.02469136+0.j,  1.24691358+0.j]])

This unitary matrix converts between states in the J and S-bases. We can represent such a state as 1D np array of complex numbers of the same size as the channel arrays, representing the coefficient for the radial wavefunction in each channel.

In [92]:
state_sbasis = np.zeros_like(ch_sbasis, dtype=complex)
state_sbasis

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

Let's set one particular channel to have some wavefunction in it, and see what our resulting $s$-basis wavefunction is:

In [93]:
state_sbasis[1] = 1
np.dot( state_sbasis.conj() , state_sbasis ) # should be 1 to be normalizable quantum state

(1+0j)

In [94]:
U_s2j @ state_sbasis

array([ 0. +0.j, -0.2+0.j,  0.8+0.j,  0. +0.j,  0. +0.j])

In [99]:
U_j2s = np.linalg.inv(U_s2j)

In [100]:
state_jbasis = np.zeros_like(ch_jbasis, dtype=complex)
state_jbasis[4] = 1j
state_jbasis

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j])

In [101]:
state_sbasis =  U_j2s @ state_jbasis
state_sbasis

array([0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+1.11111111j,
       0.+0.11111111j])

In [102]:
U_s2j @ state_sbasis

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j])